In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys

year='2022'

In [ ]:
#Read CSV file containing the persons from the WM 
df_persons_NE_inactive_20211117__dir = '/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only' # use your path
df_persons_NE_inactive_20211117_file = os.path.join(df_persons_NE_inactive_20211117__dir, "df_persons_NE_inactive_20211117.csv")
df_persons_NE_inactive_20211117 = pd.read_csv(df_persons_NE_inactive_20211117_file, index_col=None, header=0)


In [ ]:
#Read CSV file containing the households from the WM
df_households_NE_extended__dir = '/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only' # use your path
df_households_NE_extended_file = os.path.join(df_households_NE_extended__dir, "df_households_NE_clean.csv")
df_households_NE_extended = pd.read_csv(df_households_NE_extended_file, index_col=None, header=0)


In [ ]:
#Read CSV file containing the economic activities per OA level, related to inactive people (QS601EW Economic Activity)
## This dataset has been already cleaned and column names have been updated
## This dataset contains information about the number of inactive people per location (OA area).
df_QS601EW_dir = '/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/Data' # use your path
df_QS601EW_file = os.path.join(df_QS601EW_dir, "QS601EW.csv")
df_QS601EW_inactive = pd.read_csv(df_QS601EW_file, index_col=None, header=0)


In [ ]:
df_QS601EW_inactive = df_QS601EW_inactive[df_QS601EW_inactive.columns.drop(list(df_QS601EW_inactive.filter(regex='All')))]
df_QS601EW_inactive = df_QS601EW_inactive[df_QS601EW_inactive.columns.drop(list(df_QS601EW_inactive.filter(regex='Part-time')))]
df_QS601EW_inactive = df_QS601EW_inactive[df_QS601EW_inactive.columns.drop(list(df_QS601EW_inactive.filter(regex='Full-time;')))]
df_QS601EW_inactive = df_QS601EW_inactive[df_QS601EW_inactive.columns.drop(list(df_QS601EW_inactive.filter(regex='Self-employed')))]
df_QS601EW_inactive = df_QS601EW_inactive[df_QS601EW_inactive.columns.drop(list(df_QS601EW_inactive.filter(regex='Unemployed')))]

df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically active: Total; measures: Value', 'Economically_inactive_Total', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically active: Full-time student; measures: Value', 'Economically_active_Fulltime_student', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically inactive: Retired; measures: Value', 'Economically_inactive_Retired', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically inactive: Student (including full-time students); measures: Value', 'Economically_inactive_Student', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically inactive: Looking after home or family; measures: Value', 'Economically_inactive_Looking_after_home_or_family', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically inactive: Long-term sick or disabled; measures: Value', 'Economically_inactive_Longterm_sick_or_disabled', regex=False)
df_QS601EW_inactive.columns = df_QS601EW_inactive.columns.str.replace('Economic Activity: Economically inactive: Other; measures: Value', 'Economically_inactive_Other', regex=False)

df_QS601EW_inactive

In [ ]:
# Create a dataframe only containing the people that are Economic_activity = 'Inactive' and age between 16 and 64
df_persons_NE_inactive_16_64 = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >= 16) & (df_persons_NE_inactive_20211117['Age'] <= 64)]


In [ ]:
# Total number of persons that have to be assinged an inactive category:
len(df_persons_NE_inactive_16_64)

## Start selecting inactive categories (student, looking after home or family, retired, sick or other) based on sociodemographic characteristics of the persons

### Students

In [ ]:
## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of STUDENTS will be stored
persons_NE_inactive_students_chosen_all_list = []


# % inactive conversor value 2011 to 2021
## This value is converting the % of inactive people (each type) from 2011 to 2021 based on data from "Regional labour market statistics: HI05 Headline indicators for the WM"

#VALEUR it 0
#inactive_student_conversor = 1.196
#VALEUR it 1
inactive_student_conversor = 2.4


#  Relationship between male and female people per reason of inactivity (male + female =1)
## These values are coming from Regional labour market statistics:HI05 Headline indicators for the WM. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"

# Males
percentage_1_student = 0.496

# Females
percentage_2_student = 0.504


#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]

    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2021 = len(df_persons_NE_inactive_16_64_OAarea)

    
    # Calculate the ratio of people 2021 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2021_2011 = total_inactive_in_OA_2021/total_QS601EW_inactive_in_OA_2011

    else:
        ratio_people_2021_2011 = 1

    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive STUDENTS from table QS601EW
    total_QS601EW_inactive_students_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Student')]

   
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to student ("total_QS601EW_inactive_students_in_OA_2011") 
    # 2. the "inactive_student_conversor" value
    # 3. the ratio of people 2021 vs 2011 ("ratio_people_2021_2011")
    
    # New value for 2021 =  (Value from 2011 (table QS601EW)) * ("inactive_student_conversor"  value) * (ratio of people 2021 vs 2011 ("ratio_people_2021_2011")) 
    ## This value is the number of people that will be assigned "STUDENT" in the OA area selected
    inactive_students_2021 = int(round(total_QS601EW_inactive_students_in_OA_2011 * inactive_student_conversor * ratio_people_2021_2011,0)) 
    

    
    ## START IDENTIFYING THE STUDENTS
    
    for gender in gender_list:
        
        # FIRST SELECTION
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. QS420_CELL = 26  (agents living in buildings related to education)
        ## 2. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        # Select those people QS420_CELL = 26 (agents living in buildings related to education --> student accomodations (assumption))
        globals()[f"df_{gender}_QS420_CELL_26"] = df_persons_NE_inactive_16_64_OAarea.loc[(df_persons_NE_inactive_16_64_OAarea['QS420_CELL_x'] == 26) & (df_persons_NE_inactive_16_64_OAarea['Sex'] == gender)]

        # If there are people NSSEC = 9 (students):
        if len(globals()[f"df_{gender}_QS420_CELL_26"]) > 0:
            
            
            if ((int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0))) < len(globals()[f"df_{gender}_QS420_CELL_26"])):
                
                # Select randomly the number of people that will be assigned as inactive               
                globals()[f"df_{gender}_first_students_chosen"] = globals()[f"df_{gender}_QS420_CELL_26"].sample(int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0)))
                
           
            else:
                
                # Select all people that will be assigned as students
                globals()[f"df_{gender}_first_students_chosen"] = globals()[f"df_{gender}_QS420_CELL_26"]
                
                # Not all students have already been selected, just a few of them.
                # Second selection should be run
            
        # If there is not any student in the OA area:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            globals()[f"df_{gender}_first_students_chosen"] =  pd.DataFrame()
                
            # None students have already been selected.
            # Second selection should be run.
        
        
        #######################################################################################################################
        
        # SECOND SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. NSSEC == 9 (Students)
        ## 2. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously
        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        second_selection_inactive_students_2021 = int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_plus_first_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea, globals()[f"df_{gender}_first_students_chosen"]]))
              
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_second_selection = df_persons_NE_inactive_16_64_OAarea_plus_first_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select those that are students
        globals()[f"df_{gender}_NSSEC_9"] = df_persons_NE_inactive_16_64_OAarea_second_selection.loc[(df_persons_NE_inactive_16_64_OAarea_second_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_OAarea_second_selection['NSSEC'] == 9)]
        
        
        if (second_selection_inactive_students_2021 > 0):
        
            # If there are people NSSEC = 9 (students):
            if len(globals()[f"df_{gender}_NSSEC_9"]) > 0:
            
                if ((second_selection_inactive_students_2021) < len(globals()[f"df_{gender}_NSSEC_9"])):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_second_students_chosen"] = globals()[f"df_{gender}_NSSEC_9"].sample(second_selection_inactive_students_2021)
                
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_second_students_chosen"] = globals()[f"df_{gender}_NSSEC_9"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            # If there is not any person living in multi-person households and are aged 16-35 in the OA area:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                globals()[f"df_{gender}_second_students_chosen"] =  pd.DataFrame()
                
                # None persons living in multi-person households and are aged 16-35 in the OA area have been selected
                # Third selection should be run.
                
        # if second_selection_inactive_students_2021 = 0, then no more studetns should be selected
        else:
            
            globals()[f"df_{gender}_second_students_chosen"] =  pd.DataFrame()
                    

        ################################################################################################################
        
        # THIRD SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. LC4408_C_AHTHUK11 = 5 (Domestic situation = Multi-person household)
        ## 2. Age: people 16-35 years old
        ## 3. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        third_selection_inactive_students_2021 = int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_second_selection_plus_second_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_second_selection, globals()[f"df_{gender}_second_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_third_selection = df_persons_NE_inactive_16_64_OAarea_second_selection_plus_second_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select those that LC4408_C_AHTHUK11 = 5 and aged 16-35 (specific gender and OA area)
        globals()[f"df_{gender}_16_35_AHTHUK11_5"] = df_persons_NE_inactive_16_64_OAarea_third_selection.loc[(df_persons_NE_inactive_16_64_OAarea_third_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_OAarea_third_selection['Age'] >= 16) & (df_persons_NE_inactive_16_64_OAarea_third_selection['Age'] <= 35) & (df_persons_NE_inactive_16_64_OAarea_third_selection['LC4408_C_AHTHUK11_x'] == 5)]
        

        if (third_selection_inactive_students_2021 > 0):            
                    
            # If there are people LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area):
            if len(globals()[f"df_{gender}_16_35_AHTHUK11_5"]) > 0:
            
                if (third_selection_inactive_students_2021) < len(globals()[f"df_{gender}_16_35_AHTHUK11_5"]):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_third_students_chosen"] = globals()[f"df_{gender}_16_35_AHTHUK11_5"].sort_values('Age', ascending = True).head(third_selection_inactive_students_2021)
                
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_third_students_chosen"] = globals()[f"df_{gender}_16_35_AHTHUK11_5"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            # If there is not any person living in multi-person households and are aged 16-35 in the OA area:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                globals()[f"df_{gender}_third_students_chosen"] =  pd.DataFrame()
                
                # None persons LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area) have been selected
                # Forth selection should be run.
        
        # if third_selection_inactive_students_2021 = 0, then no more studetns should be selected
        else:
            
            globals()[f"df_{gender}_third_students_chosen"] =  pd.DataFrame()  
        

        #############################################################################################################
        
        # FORTH SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. LC4408_C_AHTHUK11 = 1,2, OR 3 (Domestic situation = one person household, married, cohabitating couple)
        ## 2. Age: people 16-35 years old
        ## 3. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        # Remaining NUMBER of people to be assigned as INACTIVE:
        forth_selection_inactive_students_2021 = int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"]) -len(globals()[f"df_{gender}_third_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_third_selection_plus_third_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_third_selection, globals()[f"df_{gender}_third_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_forth_selection = df_persons_NE_inactive_16_64_OAarea_third_selection_plus_third_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        
        # Select those that LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area)
        domestic_array = [1,2,3]
        globals()[f"df_{gender}_16_35_domestic123"] = df_persons_NE_inactive_16_64_OAarea_forth_selection.loc[(df_persons_NE_inactive_16_64_OAarea_forth_selection['Sex'] == gender)  & (df_persons_NE_inactive_16_64_OAarea_forth_selection['LC4408_C_AHTHUK11_x'].isin(domestic_array)) & (df_persons_NE_inactive_16_64_OAarea_forth_selection['Age'] >= 16) & (df_persons_NE_inactive_16_64_OAarea_forth_selection['Age'] <= 35)]
        
        
        if (forth_selection_inactive_students_2021 > 0):
                
            # If there are people LC4408_C_AHTHUK11 = 1,2,3 OR 5 :
            if len(globals()[f"df_{gender}_16_35_domestic123"]) > 0:
            
                if ((forth_selection_inactive_students_2021) <= len(globals()[f"df_{gender}_16_35_domestic123"])):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_forth_students_chosen"] = globals()[f"df_{gender}_16_35_domestic123"].sort_values('Age', ascending = True).head(forth_selection_inactive_students_2021)
           
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_forth_students_chosen"] = globals()[f"df_{gender}_16_35_domestic123"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            #If the dataframe does not have any rows, then it will be empty.        
            else:
                globals()[f"df_{gender}_forth_students_chosen"] = pd.DataFrame()
        
        
        # if forth_selection_inactive_students_2021 = 0, then no more studetns should be selected
        else:
             globals()[f"df_{gender}_forth_students_chosen"] = pd.DataFrame()       
        
        
        ###################################################################################################################
        
        ## FIFTH SELECTION
        # Finally, select randomly the remaining ones (leftovers) based on the age only (sort age ascending order and selecte the remaining ones)
        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        fifth_selection_inactive_students_2021 = int(round(inactive_students_2021 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"]) - len(globals()[f"df_{gender}_third_students_chosen"]) - len(globals()[f"df_{gender}_forth_students_chosen"])

        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_forth_selection_plus_forth_students_chosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_forth_selection, globals()[f"df_{gender}_forth_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_fifth_selection = df_persons_NE_inactive_16_64_OAarea_forth_selection_plus_forth_students_chosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select only those of a specific gender
        df_persons_NE_inactive_16_64_OAarea_fifth_selection_gender = df_persons_NE_inactive_16_64_OAarea_fifth_selection.loc[(df_persons_NE_inactive_16_64_OAarea_fifth_selection['Sex'] == gender)]
        
        
        if fifth_selection_inactive_students_2021 > 0:
                
            globals()[f"df_{gender}_fifth_students_chosen"] = df_persons_NE_inactive_16_64_OAarea_fifth_selection_gender.sort_values('Age', ascending = True).head(fifth_selection_inactive_students_2021)

        
        else:
                    
            globals()[f"df_{gender}_fifth_students_chosen"] = pd.DataFrame() 
            
        
        #Concatenate the selected dataframes with the selected students
        globals()[f"df_{gender}_inactive_students"] = (pd.concat([globals()[f"df_{gender}_first_students_chosen"], globals()[f"df_{gender}_second_students_chosen"], globals()[f"df_{gender}_third_students_chosen"], globals()[f"df_{gender}_forth_students_chosen"], globals()[f"df_{gender}_fifth_students_chosen"]]))
        
        
        #Append the dataframe into the temporal list
        persons_NE_inactive_students_chosen_all_list.append(globals()[f"df_{gender}_inactive_students"])
    
    
# Concatenate the selected dataframes with the selected students
df_inactive_students_all = pd.concat(persons_NE_inactive_students_chosen_all_list, axis=0, ignore_index=True)
            
        
print('The code has finished. Check results!')       
        
        

In [ ]:
# check the total number of students selected:
len(df_inactive_students_all)

In [ ]:
# Check if there are duplicates
df_inactive_students_all_1 = df_inactive_students_all

# Remove duplicates
df_inactive_students_all_1 = df_inactive_students_all.drop_duplicates(subset='PID_AreaMSOA',keep = False)

len(df_inactive_students_all_1)

In [ ]:
#####################################################
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_students = len(df_inactive_students_all)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_students was', percentage_students)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_students_male = df_inactive_students_all.loc[(df_inactive_students_all['Sex'] == 1)]
percentage_males_students = len(df_persons_NE_inactive_students_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_students was', percentage_males_students)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_students_female = df_inactive_students_all.loc[(df_inactive_students_all['Sex'] == 2)]
percentage_females_students = len(df_persons_NE_inactive_students_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_students was', percentage_females_students)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_students_male)/(len(df_inactive_students_all)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_students_female)/(len(df_inactive_students_all)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_student_conversor" parameter value.

In [ ]:
# Save dataframe
df_inactive_students_all_export20220225_correct = df_inactive_students_all
df_inactive_students_all_export20220225_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_inactive_students_all_export_correct.csv', encoding='utf-8', header=True)


## LOOKING AFTER FAMILY / HOME

In [ ]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_plus_students = (pd.concat([df_persons_NE_inactive_16_64, df_inactive_students_all]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_NO_students = df_persons_NE_inactive_16_64_plus_students.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students)

In [ ]:
# check the remaining agents
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all)

In [ ]:
## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
## Run first females because there are more to be assinged in this category.
gender_list = [2,1]

#Create an empty list where the small blocks of dataframes of "LOOKING AFTER" people will be stored
people_NE_inactive_lookingafter_all_list = []


# % inactive conversor value 2011 to 2021
## This value is converting the % of inactive people (each type) from 2011 to 2021 based on data from "Regional labour market statistics: HI01 Headline indicators for the WM"

#VALEUR it 0
#inactive_loookingafter_conversor = 1.332
#VALEUR it 1
inactive_loookingafter_conversor = 2.2

#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI05 Headline indicators for the WM. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_lookingafter = 0.163

# Females
percentage_2_lookingafter = 0.837


#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:
    
    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
            
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2021 = len(df_persons_NE_inactive_16_64_OAarea)

    
    # Calculate the ratio of people 2021 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2021_2011 = total_inactive_in_OA_2021/total_QS601EW_inactive_in_OA_2011
    else:
        ratio_people_2021_2011 = 1
        
        
    # Get the VALUE of the number of people living in the seleted OA area that are inactive LOOKING AFTER from table QS601EW
    total_QS601EW_inactive_lookingafter_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Looking_after_home_or_family')]
    
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to looking after ("total_QS601EW_inactive_lookingafter_in_OA_2011") 
    # 2. the "inactive_lookingafter_conversor" value
    # 3. the ratio of people 2021 vs 2011 ("ratio_people_2021_2011")
    
    # New value for 2021 =  (Value from 2011 (table QS601EW)) * ("inactive_lookingafter_conversor"  value) * (ratio of people 2021 vs 2011 ("ratio_people_2021_2011")) 
    ## This value is the number of people that will be assigned "LOOKING AFTER" in the OA area selected
    inactive_lookingafter_2021 = int(round(total_QS601EW_inactive_lookingafter_in_OA_2011 * inactive_loookingafter_conversor * ratio_people_2021_2011,0)) 
    

    
    #################################################################################################################
    ## START IDENTIFYING THE "LOOKING AFTER HOME /  FAMILY" ATTRIBUTE VALUE
    
    # SELECT people as 'LOOKING AFTER HOME/FAMILY' BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. Marital status: Married or Couple
    ### 2. Children dependency: True
    ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    
    # In case there are still leftovers:
    ## Second selection:
    ### 1. Marital status: Married or Couple
    ### 2. Children dependency: False
    ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    ### 4. Age: sort age by 9asecnding order = False) and choose the top 'n values required
    
    # In case there are still leftovers:
    ## Third selection:
    ### 1. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    ### 2. Marital status: any type (married, couple or single)
    ### Age: no age requirements.
    

    #Empty list where households ID's will be included once one gender is ran
    # This is to avoid that more than one person is assigned as "looking after home/family" per household
    all_HID_AreaOA_x_list = []
    
    
    for gender in gender_list:
        
        # In order to avoid having more than one person per household in the category "LOOKING AFTER FAMILY / HOME",
        # After the first iteration (female loop) and when the second loop starts (male), 
        # If a male belongs to a household were a female was selected, then the male is not considered to be in the category "LOOKING AFTER FAMILY / HOME"
        # Create a new dataframe removing the rows which household IDs have already assigned a person as "LOOKING AFTER HOME/FAMILY"
        # The values to be removed are coming from the list "all_HID_AreaOA_x_list"
        # For females, the list is always empty
        # For males, this list contains the HID_AreaOA_x of females that were already assigned as "LOOKING AFTER HOME/FAMILY"
        # So, those males that live in those households will NOT be considered to be assigned as "LOOKING AFTER HOME/FAMILY"
        # This is to avoid that a household has more than one person in this category.
        df_persons_NE_inactive_16_64_NO_students_unique = df_persons_NE_inactive_16_64_NO_students[~df_persons_NE_inactive_16_64_NO_students.HID_AreaOA_x.isin(all_HID_AreaOA_x_list)]
        
        
        ## FIRST SELECTION:
        ### 1. Marital status: Married or Couple
        ### 2. Children dependency: True
        ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    
        # Store in an array the possible marital_status to be considered first
        marital_status_array = ['Married', 'Couple']
        
        
        
        # FRIST SELECTION: Select those people aged 16-64, married/couple with children dependency    
        df_persons_NE_inactive_16_64_married_or_couples_childrendepend = df_persons_NE_inactive_16_64_NO_students_unique.loc[(df_persons_NE_inactive_16_64_NO_students_unique['Marital_status'].isin(marital_status_array)) & (df_persons_NE_inactive_16_64_NO_students_unique['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_unique['Children_dependency'] == True) & (df_persons_NE_inactive_16_64_NO_students_unique['Sex'] == gender)]
        
        if (len(df_persons_NE_inactive_16_64_married_or_couples_childrendepend) > 0):
            
            if ((int(round(inactive_lookingafter_2021 * globals()[f"percentage_{gender}_lookingafter"],0))) <= len(df_persons_NE_inactive_16_64_married_or_couples_childrendepend)):
                
                # Select randomly the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME'               
                df_first_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_childrendepend.sample(int(round(inactive_lookingafter_2021 * globals()[f"percentage_{gender}_lookingafter"],0)))
                
        
            else:
                
                # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                df_first_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_childrendepend
                
                # Not all 'LOOKING AFTER FAMILY / HOME' people have already been selected, just a few of them.
                # Second selection should be run                                                                                                       
                                                                                                                     
        # If there is not any person with the selected charateristics in the dataframe:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_first_selection_lookingafter =  pd.DataFrame()
                
            # None persons have already been selected.
            # Second selection should be run.

                                                                                                                        
        ## SECOND SELECTION:                                                                                                               
        ### 1. Marital status: Married or Couple
        ### 2. Children dependency: False
        ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
        ### 4. Age: sort age by 9asecnding order = False) and choose the top 'n values required
        
        # Remaining NUMBER of people to be assigned as "LOOKING AFTER FAMILY / HOME":
        second_selection_lookingafter_2021 = (int(round(inactive_lookingafter_2021 * globals()[f"percentage_{gender}_lookingafter"],0)) - len(df_first_selection_lookingafter))
        
        # Concatenate 
        df_persons_NE_inactive_16_64_NO_students_plus_first_selection_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students, df_first_selection_lookingafter]))
                
        # Remove duplicates
        df_persons_NE_inactive_16_64_NO_students_second_selection = df_persons_NE_inactive_16_64_NO_students_plus_first_selection_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
                                                                                                                        
                                                                                                                        
        # Select those that are aged 16-64 and married or couple without any children dependency                                                                                                                     
        df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend = df_persons_NE_inactive_16_64_NO_students_second_selection.loc[(df_persons_NE_inactive_16_64_NO_students_second_selection['Marital_status'].isin(marital_status_array)) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Children_dependency'] == False)]
                                                                                                                         
                                                                                                                      
        if (second_selection_lookingafter_2021 > 0):
        
            if (len(df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend) > 0):
                                                                                                                        
                if ((second_selection_lookingafter_2021) <= len(df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend)):
                
                    # Select the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME' sorted by age              
                    df_second_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend.sort_values('Age', ascending = False).head(second_selection_lookingafter_2021)
                                                                                            
                else:
                
                    # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                    df_second_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend
                                                                                                                        
            # If there is not any person with the selected charateristics in the dataframe:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                df_second_selection_lookingafter =  pd.DataFrame()                                                                                                            
                                                                                                                        
                                                                                                                        
        # If there remaining number of people to be selected (second_selection_lookingafter_2021) is zero:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_second_selection_lookingafter =  pd.DataFrame() 
                                                                                                                        
                                                                                                                        
                                                                                                                        
        ## THIRD SELECTION                                                                                                                
        ### 1. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
        ### 2. Marital status: any type (married, couple or single)
        ### Age: no age requirements.
                                                                                                                        
        # Remaining NUMBER of people to be assigned as "LOOKING AFTER FAMILY / HOME":
        third_selection_lookingafter_2021 = (int(round(inactive_lookingafter_2021 * globals()[f"percentage_{gender}_lookingafter"],0)) - len(df_first_selection_lookingafter) - len(df_second_selection_lookingafter))
        
        # Concatenate 
        df_persons_NE_inactive_16_64_NO_students_second_selection_plus_second_selection_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students_second_selection, df_second_selection_lookingafter]))
                
        # Remove duplicates
        df_persons_NE_inactive_16_64_NO_students_third_selection = df_persons_NE_inactive_16_64_NO_students_second_selection_plus_second_selection_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
                                                                                                                   
        # Select those that are aged 16-64 (no marital status and age requirements)                                                                                                                        
        df_persons_NE_inactive_16_64_remaining = df_persons_NE_inactive_16_64_NO_students_third_selection.loc[(df_persons_NE_inactive_16_64_NO_students_third_selection['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_third_selection['Sex'] == gender)]
                                                                                                                          
                                                                                                                        
        if (third_selection_lookingafter_2021 > 0): 
           
            if (len(df_persons_NE_inactive_16_64_remaining) > 0): 
                                                                                                                        
                if ((third_selection_lookingafter_2021) <= len(df_persons_NE_inactive_16_64_remaining)):
                
                    # Select the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME' based on the value stored in "third_selection_lookingafter_2021"          
                    df_third_selection_lookingafter = df_persons_NE_inactive_16_64_remaining.sample(third_selection_lookingafter_2021)
                                                                                                                                 
                else:
                
                    # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                    df_third_selection_lookingafter = df_persons_NE_inactive_16_64_remaining   
                                                                                                                        
            # If there is not any person with the selected charateristics in the dataframe:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                df_third_selection_lookingafter =  pd.DataFrame() 
                                                                                                                        
        # If there remaining number of people to be selected (third_selection_lookingafter_2021) is zero:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_third_selection_lookingafter =  pd.DataFrame()                                                                                                                 
                                                                                                                        
                                                                                                                        
                                                                                                                        
        #Concatenate the selected dataframes with the selected Looking after
        globals()[f"df_{gender}_inactive_looking_after"] = (pd.concat([df_first_selection_lookingafter, df_second_selection_lookingafter, df_third_selection_lookingafter]))
        
        
        # Include in the list the HID_AreaOA_x values in order to avoid that another person of the other gender can be categorised as "Looking after family/home"
        #Append the dataframe into the temporal list
        if len(globals()[f"df_{gender}_inactive_looking_after"]) > 0:
            all_HID_AreaOA_x_list = globals()[f"df_{gender}_inactive_looking_after"]['HID_AreaOA_x'].tolist()
        

        people_NE_inactive_lookingafter_all_list.append(globals()[f"df_{gender}_inactive_looking_after"])
                                                                                                                 

# Concatenate the selected dataframes with the selected looking after family/home people
df_inactive_lookingafter_all_22feb_3 = pd.concat(people_NE_inactive_lookingafter_all_list, axis=0, ignore_index=True)
                                                                                                                        
                                                                                                                     
print('Code has finished. Check the results!')  

In [ ]:
# check the number of peoples selected as inactive looking after family/home
len(df_inactive_lookingafter_all_22feb_3)

In [ ]:
# check if there are duplicates
df_inactive_lookingafter_all_22feb_3_1 = df_inactive_lookingafter_all_22feb_3

# Remove duplicates
df_inactive_lookingafter_all_22feb_3_1 = df_inactive_lookingafter_all_22feb_3_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)

len(df_inactive_lookingafter_all_22feb_3_1)

In [ ]:
#####################################################
# df_inactive_lookingafter_all_22feb_3
##   CHECKS:
    
# Calculate the % of lookingafter within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_lookingafter = len(df_inactive_lookingafter_all_22feb_3)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_lookingafter was', percentage_lookingafter)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are lookingafter
df_persons_NE_inactive_lookingafter_male = df_inactive_lookingafter_all_22feb_3.loc[(df_inactive_lookingafter_all_22feb_3['Sex'] == 1)]
percentage_males_lookingafter = len(df_persons_NE_inactive_lookingafter_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_lookingafter was', percentage_males_lookingafter)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are lookingafter
df_persons_NE_inactive_lookingafter_female = df_inactive_lookingafter_all_22feb_3.loc[(df_inactive_lookingafter_all_22feb_3['Sex'] == 2)]
percentage_females_lookingafter = len(df_persons_NE_inactive_lookingafter_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_lookingafter was', percentage_females_lookingafter)
    

# Calculate the % proportion of males that are lookingafter
percentage_proportion_male = len(df_persons_NE_inactive_lookingafter_male)/(len(df_inactive_lookingafter_all_22feb_3)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are lookingafter
percentage_proportion_female = len(df_persons_NE_inactive_lookingafter_female)/(len(df_inactive_lookingafter_all_22feb_3)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_loookingafter_conversor" parameter value.

In [ ]:
# Save dataframe
df_inactive_lookingafter_all_22feb_3_export20220225_correct = df_inactive_lookingafter_all_22feb_3
df_inactive_lookingafter_all_22feb_3_export20220225_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_inactive_lookingafter_all_correct.csv', encoding='utf-8', header=True)


## Retired

In [ ]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS AND NOT LOOKING AFTER
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_NO_students_plus_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students, df_inactive_lookingafter_all_22feb_3]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter = df_persons_NE_inactive_16_64_NO_students_plus_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter)

In [ ]:
# Check the number of remaining people to be assigned as "Retired", "Sick" or "Other"
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3)

In [ ]:
# Calculate the differences between 
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter)

In [ ]:
# Start developing the code to identify INACTIVE RETIRED

## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of "RETIIRED" people will be stored
people_NE_inactive_retired_all_list = []


# % inactive conversor value 2011 to 2021
## This value is converting the % of inactive people (each type) from 2011 to 2021 based on data from "Regional labour market statistics: HI05 Headline indicators for the WM"
inactive_retired_conversor = 0.758


#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI05 Headline indicators for the WM. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_retired = 0.439

# Females
percentage_2_retired = 0.561




#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2021 = len(df_persons_NE_inactive_16_64_OAarea)
    
    
    # Calculate the ratio of people 2021 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2021_2011 = total_inactive_in_OA_2021/total_QS601EW_inactive_in_OA_2011
        #print('Ratio people 2021_2011')
        #print(ratio_people_2021_2011)
    else:
        ratio_people_2021_2011 = 1
    
    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive RETIRED from table QS601EW
    total_QS601EW_inactive_retired_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Retired')]

    
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to each category (student, looking after, retired...)
    # 2. the inactive_XXXX_conversor
    # 3. the ratio of people 2021 vs 2011 ("ratio_people_2021_2011") [CONSTANT VALUE FOR ALL CATEGORIES]
    

    # New value for 2021 =  (Value from 2011 (table QS601EW)) * ("inactive_retired_conversor"  value) * (ratio of people 2021 vs 2011 ("ratio_people_2021_2011")) 
    ## This value is the number of people that will be assigned "RETIRED" in the OA area selected
    inactive_retired_2021 = int(round(total_QS601EW_inactive_retired_in_OA_2011 * inactive_retired_conversor * ratio_people_2021_2011,0)) 

    
    #################################################################################################################
    ## START IDENTIFYING THE "RETIRED ATTRIBUTE VALUE
    
    
    # SELECT people as 'RETIRED' BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. People aged 60-64
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    # In case there are still leftovers:
    ## Second selection:
    ### 1. People aged 55-59
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    ## Third selection:
    ### 1. People aged 50-54
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    ## Forth selection:
    ### 1. People aged < 50
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously

    for gender in gender_list:
        
        # Dataframe for First selection
        df_people_inactive_60_64_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 60) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Second selection
        df_people_inactive_55_59_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 55) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] <= 59) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Third selection
        df_people_inactive_50_54_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] <= 54) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Forth selection
        df_people_inactive_50less_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] < 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        
        # Start choosing people in each of the 4 selection options defined before AND considering the gender %
        ## males: 54.5%   = percentage_1_retired
        ## females: 45.5% = percentage_2_retired
        

        if (len(df_people_inactive_60_64_gender) > int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))):
        
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender.sample(int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0)))

            df_second_selection_retired = pd.DataFrame()
            df_third_selection_retired = pd.DataFrame()
            df_forth_selection_retired = pd.DataFrame()
            
        elif (len(df_people_inactive_60_64_gender) == int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))):
            
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender

            df_second_selection_retired = pd.DataFrame()
            df_third_selection_retired = pd.DataFrame()
            df_forth_selection_retired = pd.DataFrame()
            
            
        else:
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender

            # There are still remaining some people to be assigned as "RETIRED"
            if (len(df_people_inactive_55_59_gender) > (int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired)):
        
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender.sample(int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired))

                df_third_selection_retired = pd.DataFrame()
                df_forth_selection_retired = pd.DataFrame()
            
            elif (len(df_people_inactive_55_59_gender) == (int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired)):
        
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender

                df_third_selection_retired = pd.DataFrame()
                df_forth_selection_retired = pd.DataFrame()
            
            
            else:
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender

                # There are still remaining some people to be assigned as "RETIRED"
                if (len(df_people_inactive_50_54_gender) > (int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired) - len(df_second_selection_retired)):
        
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender.sample(int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired) - len(df_second_selection_retired))

                    df_forth_selection_retired = pd.DataFrame()
            
                elif (len(df_people_inactive_50_54_gender) == (int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired) - len(df_second_selection_retired)):
        
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender

                    df_forth_selection_retired = pd.DataFrame()
            
                else:
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender

                    
                    # Forth and final selection:                     
                    df_forth_selection_retired = df_people_inactive_50less_gender.sort_values('Age', ascending = False).head(int(round(inactive_retired_2021 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired) - len(df_second_selection_retired) - len(df_third_selection_retired))

        
        # Once all possible ways to select people have been computed, there is a need to concatenate all dataframes:
        df_people_NE_inactive_retired = (pd.concat([df_first_selection_retired, df_second_selection_retired, df_third_selection_retired, df_forth_selection_retired]))
 
        #Append the dataframe into the temporal list
        people_NE_inactive_retired_all_list.append(df_people_NE_inactive_retired)
        

#concatenate all people "retired" in one dataframe
df_people_NE_inactive_retired_all_v2 = pd.concat(people_NE_inactive_retired_all_list, axis=0, ignore_index=True)



print('Job done, Check the results')



In [ ]:
# Check the amount of people selected as retired:
len(df_people_NE_inactive_retired_all_v2)

In [ ]:
#####################################################
# df_people_NE_inactive_retired_all_v2
##   CHECKS:
    
# Calculate the % of retired within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_retired = len(df_people_NE_inactive_retired_all_v2)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_retired was', percentage_retired)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are retired
df_persons_NE_inactive_retired_male = df_people_NE_inactive_retired_all_v2.loc[(df_people_NE_inactive_retired_all_v2['Sex'] == 1)]
percentage_males_retired = len(df_persons_NE_inactive_retired_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_retired was', percentage_males_retired)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are retired
df_persons_NE_inactive_retired_female = df_people_NE_inactive_retired_all_v2.loc[(df_people_NE_inactive_retired_all_v2['Sex'] == 2)]
percentage_females_retired = len(df_persons_NE_inactive_retired_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_retired was', percentage_females_retired)
    

# Calculate the % proportion of males that are retired
percentage_proportion_male = len(df_persons_NE_inactive_retired_male)/(len(df_people_NE_inactive_retired_all_v2)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are retired
percentage_proportion_female = len(df_persons_NE_inactive_retired_female)/(len(df_people_NE_inactive_retired_all_v2)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_retired_conversor" parameter value.

In [ ]:
# Check if there are duplicates:
df_people_NE_inactive_retired_all_v2_1 = df_people_NE_inactive_retired_all_v2

# Remove duplicates
df_people_NE_inactive_retired_all_v2_1 = df_people_NE_inactive_retired_all_v2_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_people_NE_inactive_retired_all_v2_1)

In [ ]:
# Save dataframe
df_people_NE_inactive_retired_all_v2_export_20220302_correct = df_people_NE_inactive_retired_all_v2
df_people_NE_inactive_retired_all_v2_export_20220302_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_people_NE_inactive_retired_all_correct.csv', encoding='utf-8', header=True)


## Sick

In [ ]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS, NOT RETIRED AND NOT LOOKING AFTER
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_plus_retired = (pd.concat([df_persons_NE_inactive_16_64_NO_students_NO_lookingafter, df_people_NE_inactive_retired_all_v2]))

# Remove duplicates
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_plus_retired.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired)

In [ ]:
# Check the number of remaining people to be assigned as "Retired", "Sick" or "Other"

len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_people_NE_inactive_retired_all_v2)

In [ ]:
# Calculate the differences between 
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired) - len(df_people_NE_inactive_retired_all_v2)

In [ ]:
# Start developing the code to identify LONGTERM SICK people

## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of "LONGTERM SICK" people will be stored
people_NE_inactive_longtermsick_all_list = []


# % inactive conversor value 2011 to 2021
## This value is converting the % of inactive people (each type) from 2011 to 2021 based on data from "Regional labour market statistics: HI05 Headline indicators for the WM"
#inactive_longtermsick_conversor = 1.074
inactive_longtermsick_conversor = 6


#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI05 Headline indicators for the WM . Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_longtermsick = 0.505

# Females
percentage_2_longtermsick = 0.495




#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2021 = len(df_persons_NE_inactive_16_64_OAarea)

    
    
    # Calculate the ratio of people 2021 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2021_2011 = total_inactive_in_OA_2021/total_QS601EW_inactive_in_OA_2011
        #print('Ratio people 2021_2011')
        #print(ratio_people_2021_2011)
    else:
        ratio_people_2021_2011 = 1
    
    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive LONGTERMSICK from table QS601EW
    total_QS601EW_inactive_longtermsick_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Longterm_sick_or_disabled')]


    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to each category (student, looking after, retired...)
    # 2. the inactive_XXXX_conversor
    # 3. the ratio of people 2021 vs 2011 ("ratio_people_2021_2011") [CONSTANT VALUE FOR ALL CATEGORIES]
    

    # New value for 2021 =  (Value from 2011 (table QS601EW)) * ("inactive_longtermsick_conversor"  value) * (ratio of people 2021 vs 2011 ("ratio_people_2021_2011")) 
    ## This value is the number of people that will be assigned "LONG TERM SICK" in the OA area selected
    inactive_longtermsick_2021 = int(round(total_QS601EW_inactive_longtermsick_in_OA_2011 * inactive_longtermsick_conversor * ratio_people_2021_2011,0)) 

    

    #################################################################################################################
    ## START IDENTIFYING THE "LONGTERM SICK" ATTRIBUTE VALUE
    
    
    # SELECT people as "LONGTERM SICK" BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. People aged 50-64
    ### 2. Sex: proportions based on variables "percentage_1_longtermsick" and "percentage_2_longtermsick" assigned previously
    # In case there are still leftovers:
    ## Second selection:
    ### 1. People aged < 50
    ### 2. Sex: proportions based on variables "percentage_1_longtermsick" and "percentage_2_longtermsick" assigned previously

    for gender in gender_list:
        
        # Dataframe for First selection
        ## people of a specific sex, age >=50 and specific OA area
        df_people_inactive_longtermsick_50_64_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Age'] >= 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Area_OA_x'] == OA_area)]
        
        # Dataframe for Second selection
        ## people of a specific sex, age >50 and specific OA area
        df_people_inactive_longtermsick_50less_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Age'] < 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Area_OA_x'] == OA_area)]
        

        if (len(df_people_inactive_longtermsick_50_64_gender) > int(round(inactive_longtermsick_2021 * globals()[f"percentage_{gender}_longtermsick"],0))):
        
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender.sample(int(round(inactive_longtermsick_2021 * globals()[f"percentage_{gender}_longtermsick"],0)))

            df_second_selection_longtermsick = pd.DataFrame()
            
        elif (len(df_people_inactive_longtermsick_50_64_gender) == int(round(inactive_longtermsick_2021 * globals()[f"percentage_{gender}_longtermsick"],0))):
            
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender

            df_second_selection_longtermsick = pd.DataFrame()
            
            
        else:
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender

            # There are still remaining some people to be assigned as "LONGTERM SICK"
            if (len(df_people_inactive_longtermsick_50less_gender) > int(round(inactive_longtermsick_2021 * globals()[f"percentage_{gender}_longtermsick"],0) - len(df_first_selection_longtermsick))):

                ## Second selection:
                df_second_selection_longtermsick = df_people_inactive_longtermsick_50less_gender.sample(int(round(inactive_longtermsick_2021 * globals()[f"percentage_{gender}_longtermsick"],0)) - len(df_first_selection_longtermsick))

            else:
                
                ## Second selection:
                df_second_selection_longtermsick = df_people_inactive_longtermsick_50less_gender

    
        # Once all possible ways to select people have been computed, there is a need to concatenate all dataframes:
        df_people_NE_inactive_longtermsick = (pd.concat([df_first_selection_longtermsick, df_second_selection_longtermsick]))
 
        #Append the dataframe into the temporal list
        people_NE_inactive_longtermsick_all_list.append(df_people_NE_inactive_longtermsick)
        

#concatenate all people "LONGTERM SICK" in one dataframe
df_people_NE_inactive_longtermsick_all = pd.concat(people_NE_inactive_longtermsick_all_list, axis=0, ignore_index=True)



print('Job done, Check the results')

In [ ]:
# Check the number of people selected as inactive long time sick:
len(df_people_NE_inactive_longtermsick_all)

In [ ]:
# Check if there are duplicates:
df_people_NE_inactive_longtermsick_all_1 = df_people_NE_inactive_longtermsick_all

# Remove duplicates
df_people_NE_inactive_longtermsick_all_1 = df_people_NE_inactive_longtermsick_all_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_people_NE_inactive_longtermsick_all_1)

In [ ]:
#####################################################
# df_people_NE_inactive_longtermsick_all
##   CHECKS:
    
# Calculate the % of sick within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_sick = len(df_people_NE_inactive_longtermsick_all)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_sick was', percentage_sick)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are sick
df_persons_NE_inactive_sick_male = df_people_NE_inactive_longtermsick_all.loc[(df_people_NE_inactive_longtermsick_all['Sex'] == 1)]
percentage_males_sick = len(df_persons_NE_inactive_sick_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_sick was', percentage_males_sick)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are sick
df_persons_NE_inactive_sick_female = df_people_NE_inactive_longtermsick_all.loc[(df_people_NE_inactive_longtermsick_all['Sex'] == 2)]
percentage_females_sick = len(df_persons_NE_inactive_sick_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_sick was', percentage_females_sick)
    

# Calculate the % proportion of males that are sick
percentage_proportion_male = len(df_persons_NE_inactive_sick_male)/(len(df_people_NE_inactive_longtermsick_all)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are sick
percentage_proportion_female = len(df_persons_NE_inactive_sick_female)/(len(df_people_NE_inactive_longtermsick_all)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_longtermsick_conversor" parameter value.

In [ ]:
# Save dataframe
df_people_NE_inactive_longtermsick_all_export_20220302_correct = df_people_NE_inactive_longtermsick_all
df_people_NE_inactive_longtermsick_all_export_20220302_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_people_NE_inactive_longtermsick_all_correct.csv', encoding='utf-8', header=True)


## Other

In [ ]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT ASSIGNED
# Concatenate the dataframe of all inactive people (16-64) 
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired_plus_sick = (pd.concat([df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired, df_people_NE_inactive_longtermsick_all]))

# Remove duplicates
df_persons_NE_inactive_16_64_Other = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired_plus_sick.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_Other)

In [ ]:
##   CHECKS:
    
# Calculate the % of other within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_other = len(df_persons_NE_inactive_16_64_Other)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_other was', percentage_other)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are other
df_persons_NE_inactive_Other_male = df_persons_NE_inactive_16_64_Other.loc[(df_persons_NE_inactive_16_64_Other['Sex'] == 1)]
percentage_males_other = len(df_persons_NE_inactive_Other_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_other was', percentage_males_other)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are other
df_persons_NE_inactive_Other_female = df_persons_NE_inactive_16_64_Other.loc[(df_persons_NE_inactive_16_64_Other['Sex'] == 2)]
percentage_females_other = len(df_persons_NE_inactive_Other_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_other was', percentage_females_other)
    

# Calculate the % proportion of males that are other
percentage_proportion_male = len(df_persons_NE_inactive_Other_male)/(len(df_persons_NE_inactive_16_64_Other)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are other
percentage_proportion_female = len(df_persons_NE_inactive_Other_female)/(len(df_persons_NE_inactive_16_64_Other)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


In [ ]:
# Save dataframe
df_persons_NE_inactive_16_64_Other_all_export_20220302_correct = df_persons_NE_inactive_16_64_Other
df_persons_NE_inactive_16_64_Other_all_export_20220302_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_persons_NE_inactive_16_64_Other_all_correct.csv', encoding='utf-8', header=True)


In [ ]:
# Check the total number of people selected as student, looking after, retired, sick or other
a = len(df_inactive_students_all) + len(df_inactive_lookingafter_all_22feb_3) + len(df_people_NE_inactive_retired_all_v2) + len(df_people_NE_inactive_longtermsick_all) + len(df_persons_NE_inactive_16_64_Other)

In [ ]:
# Check the total number to be assigned an inactive category
b = len(df_persons_NE_inactive_16_64)

In [ ]:
# Number of people that were not assigned an inactive category:
a-b

In [ ]:
# Create a dataframe containing only the people that are Economic_activity = 'Inactive' and age 65 or more
## All these people will be considered as RETIRED ("Occupation" attribute value will be RETIRED)
df_persons_NE_inactive_65_120 = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >= 65)]


In [ ]:
len(df_persons_NE_inactive_65_120)

In [ ]:
# Save dataframe
df_persons_NE_inactive_65_120_export_20220304_correct = df_persons_NE_inactive_65_120
df_persons_NE_inactive_65_120_export_20220304_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_persons_NE_inactive_65_120_correct.csv', encoding='utf-8', header=True)


### Update the 'Occupation' column value in each of the sub-dataframes generated

In [ ]:
df_inactive_students_all['Occupation'] = 'Student'

In [ ]:
df_inactive_lookingafter_all_22feb_3['Occupation'] = 'Looking after home family'

In [ ]:
df_people_NE_inactive_retired_all_v2['Occupation'] = 'Retired'

In [ ]:
df_people_NE_inactive_longtermsick_all['Occupation'] = 'Long term sick'

In [ ]:
df_persons_NE_inactive_16_64_Other['Occupation'] = 'Other'

In [ ]:
df_persons_NE_inactive_65_120['Occupation'] = 'Retired'

### concatenate all dataframe in a single dataframe

In [ ]:
df_persons_NE_inactive_5categories = (pd.concat([df_inactive_students_all, df_inactive_lookingafter_all_22feb_3, df_people_NE_inactive_retired_all_v2, df_people_NE_inactive_longtermsick_all, df_persons_NE_inactive_16_64_Other, df_persons_NE_inactive_65_120]))


In [ ]:
# Save dataframe
df_persons_NE_inactive_5categories_export_20220304_correct = df_persons_NE_inactive_5categories
df_persons_NE_inactive_5categories_export_20220304_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_persons_NE_inactive_5categories_export_correct.csv', encoding='utf-8', header=True)
